In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from art.defences.preprocessor.feature_squeezing import FeatureSqueezing

feature_squeezer = FeatureSqueezing(clip_values=(0, 1), bit_depth=1)

def preprocess(inputs, feature_squeezer):
    inputs_squeezed, _ = feature_squeezer(inputs)
    return torch.tensor(inputs_squeezed, dtype=torch.float32)


In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./FeatureSqueezing.csv", mode='a', index=False)




In [4]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    X_test_tensor = preprocess(X_test, feature_squeezer).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./FeatureSqueezing.csv", mode='a', index=False, header=False)




In [5]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     X_test_tensor = preprocess(X_test, feature_squeezer).to(device)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def5/y_pred_{attack_name}{eps}_Def5.npy", all_preds)


In [6]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [8]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [9]:
# x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
x_train_tensor = preprocess(x_train, feature_squeezer).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

# x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
x_val_tensor = preprocess(x_val, feature_squeezer).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [10]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [11]:
# for epoch in range(50):
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = loss_function(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break

In [12]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/preprocessor/dnn_pytorch_FeatureSqueezing.pt"))

/tmp/ipykernel_3565/2979463627.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/pr

<All keys matched successfully>

In [13]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'Baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./FeatureSqueezing.csv", mode='a', index=False, header=False)

In [15]:
# percentage = ["100", "50", "20", "1"]
# model_name = ["XGB", "RF", "DT" ]

percentage = ["10"]
model_name = ["RF"]

In [16]:
epsilon = 0
Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(27, 56) (27,)
Accuracy: 0.9259259259259259

macro
Precision: 0.9230769230769231
Recall: 0.9375
F1 Score: 0.925

weighted
Precision: 0.9373219373219372
Recall: 0.9259259259259259
F1 Score: 0.9265432098765432

start RF 10 BIM
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 FGSM
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 PGD
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 DF
(106, 56) (106,)
Accuracy: 0.7547169811320755

macro
Precision: 0.7949134199134199
Recall: 0.7027529761904762
F1 Score: 0.7090371621621622

weighted
Precision: 0.7806501674426203
Recall: 0.7547169811320755
F1 Score: 0.7329646863844977

start RF 10 AutoPGD
(81, 56) (81,)
Accuracy: 0.9629629629629629

macro
Precision: 0.95945

In [17]:
epsilon = 0
Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(31, 56) (31,)
Accuracy: 0.8064516129032258

macro
Precision: 0.8125
Recall: 0.8571428571428572
F1 Score: 0.8012820512820513

weighted
Precision: 0.8790322580645161
Recall: 0.8064516129032258
F1 Score: 0.8126550868486353

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_BIM_Def5.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_FGSM_Def5.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_PGD_Def5.npy not found
start RF 10 DF
(72, 56) (72,)
Accuracy: 0.6805555555555556

macro
Precision: 0.6933497536945813
Recall: 0.6259021651964716
F1 Score: 0.6186967533962698

weighted
Precision: 0.6892788724685277
Recall: 0.6805555555555556
F1 Score: 0.6485596234042009

start RF 10 AutoPGD
(93, 56) (93,)
Accuracy: 0.806451612903225

In [18]:
epsilon = 0
Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start RF 10 baseline
(16, 56) (16,)
Accuracy: 0.875

macro
Precision: 0.9285714285714286
Recall: 0.75
F1 Score: 0.7948717948717949

weighted
Precision: 0.8928571428571428
Recall: 0.875
F1 Score: 0.8589743589743589

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_BIM_Def5.npy not found
start RF 10 FGSM
(2, 56) (2,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_PGD_Def5.npy not found
start RF 10 DF
(135, 56) (135,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.43703703703703706

macro
Precision: 0.696
Recall: 0.5581395348837209
F1 Score: 0.3857758620689655

weighted
Precision: 0.7793185185185185
Recall: 0.43703703703703706
F1 Score: 0.3371434653043848

start RF 10 AutoPGD
(48, 56) (48,)
Accuracy: 0.875

macro
Precision: 0.9285714285714286
Recall: 0.75
F1 Score: 0.7948717948717949

weighted
Precision: 0.8928571428571428
Recall: 0.875
F1 Score: 0.858974358974359

start RF 10 ZOO
(48, 56) (48,)
Accuracy: 0.875

macro
Precision: 0.9285714285714286
Recall: 0.75
F1 Score: 0.7948717948717949

weighted
Precision: 0.8928571428571428
Recall: 0.875
F1 Score: 0.858974358974359

start RF 10 CaFA
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_CaFA_Def5.npy not found
start RF 10 SINIFGSM
(4, 56) (4,)
Accuracy: 0.75

macro
Precision: 0.5
Recall: 0.375
F1 Score: 0.42857142857142855

weighted
Precision: 1.0
Recall: 0.75
F1 Score: 0.8571428571428571

start RF 10 VNIFGSM
/home/jovyan/S

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5833333333333334

macro
Precision: 0.5889877641824249
Recall: 0.6000000000000001
F1 Score: 0.5737425404944586

weighted
Precision: 0.6455320726733407
Recall: 0.5833333333333334
F1 Score: 0.5950554134697357

start RF 10 FGSM
(345, 56) (345,)
Accuracy: 0.736231884057971

macro
Precision: 0.4938058196485163
Recall: 0.4810405643738977
F1 Score: 0.4634158847356817

weighted
Precision: 0.8824571292573247
Recall: 0.736231884057971
F1 Score: 0.7994453473155746

start RF 10 PGD
(60, 56) (60,)
Accuracy: 0.5833333333333334

macro
Precision: 0.5889877641824249
Recall: 0.6000000000000001
F1 Score: 0.5737425404944586

weighted
Precision: 0.6455320726733407
Recall: 0.5833333333333334
F1 Score: 0.5950554134697357

start RF 10 DF
(58, 56) (58,)
Accuracy: 0.8620689655172413

macro
Precision: 0.9298245614035088
Recall: 0.5555555555555556
F1 Score: 0.5622641509433962

weighted
Precision: 0.8814277071990321
Recall: 0.8620689655172413
F1 Score: 0.8121014964216006

start RF 10 AutoPGD
(46, 56) (4

In [15]:
epsilon = 0

Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9642857142857143

macro
Precision: 0.975
Recall: 0.9444444444444444
F1 Score: 0.9577677224736048

weighted
Precision: 0.9660714285714286
Recall: 0.9642857142857143
F1 Score: 0.9636931695755225

start BIM
Accuracy: 0.5029585798816568

macro
Precision: 0.5107969244035643
Recall: 0.5106164988517929
F1 Score: 0.5024315869719513

weighted
Precision: 0.5154418693054353
Recall: 0.5029585798816568
F1 Score: 0.5008985166891716

start FGSM
Accuracy: 0.4677871148459384

macro
Precision: 0.510509190044851
Recall: 0.5076700898587934
F1 Score: 0.44713419405320814

weighted
Precision: 0.5223640067230833
Recall: 0.4677871148459384
F1 Score: 0.4312703563428501

start PGD
Accuracy: 0.5029585798816568

macro
Precision: 0.5107969244035643
Recall: 0.5106164988517929
F1 Score: 0.5024315869719513

weighted
Precision: 0.5154418693054353
Recall: 0.5029585798816568
F1 Score: 0.5008985166891716

start DF
Accuracy: 0.6181015452538632

macro
Precision: 0.6877251672671127
Recall: 0.662124

In [19]:
epsilon = 0
Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(599, 56) (599,)
Accuracy: 0.7679465776293823

macro
Precision: 0.5768144193010707
Recall: 0.8289776164549305
F1 Score: 0.5671087726230523

weighted
Precision: 0.9528235016851309
Recall: 0.7679465776293823
F1 Score: 0.8334157640652693

start RF 10 BIM
(268, 56) (268,)
Accuracy: 0.332089552238806

macro
Precision: 0.483695652173913
Recall: 0.16792452830188678
F1 Score: 0.24929971988795518

weighted
Precision: 0.9565622972096041
Recall: 0.332089552238806
F1 Score: 0.49301810276349345

start RF 10 FGSM
(234, 56) (234,)
Accuracy: 0.5470085470085471

macro
Precision: 0.48484848484848486
Recall: 0.2782608695652174
F1 Score: 0.35359116022099446

weighted
Precision: 0.9531209531209531
Recall: 0.5470085470085471
F1 Score: 0.6950937337677668

start RF 10 PGD
(268, 56) (268,)
Accuracy: 0.332089552238806

macro
Precision: 0.483695652173913
Recall: 0.16792452830188678
F1 Score: 0.24929971988795518

weighted
Precision: 0.9565622972096041
Recall: 0.332089552238806
F1 Score: 0.493

In [20]:
epsilon = 0
Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(61, 56) (61,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 BIM
(21, 56) (21,)
Accuracy: 0.3333333333333333

macro
Precision: 0.5882352941176471
Recall: 0.6111111111111112
F1 Score: 0.3318181818181818

weighted
Precision: 0.8823529411764707
Recall: 0.3333333333333333
F1 Score: 0.35454545454545455

start RF 10 FGSM
(22, 56) (22,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 PGD
(21, 56) (21,)
Accuracy: 0.3333333333333333

macro
Precision: 0.5882352941176471
Recall: 0.6111111111111112
F1 Score: 0.3318181818181818

weighted
Precision: 0.8823529411764707
Recall: 0.3333333333333333
F1 Score: 0.35454545454545455

start RF 10 DF
(22, 56) (22,)
Accuracy: 0.8181818181818182

macro
Precision: 0.75
Recall: 0.8888888888888888
F1 Score: 0.7708333333333333

weighted
Precision: 0.9090909090909091
Recall: 0.81818181818181

In [21]:
epsilon = 0

Def = "Def5"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]


active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
Accuracy: 0.5206611570247934

macro
Precision: 0.6499237107110162
Recall: 0.6736656062212796
F1 Score: 0.5188124582183445

weighted
Precision: 0.8191643737847304
Recall: 0.5206611570247934
F1 Score: 0.5344647747796117

start RF 10 BIM
Accuracy: 0.23657838224767358

macro
Precision: 0.2363895960214602
Recall: 0.2367948797284276
F1 Score: 0.23642187997287836

weighted
Precision: 0.236332139343917
Recall: 0.23657838224767358
F1 Score: 0.2362849404824325

start RF 10 FGSM
Accuracy: 0.25

macro
Precision: 0.30357142857142855
Recall: 0.25757575757575757
F1 Score: 0.2401883830455259

weighted
Precision: 0.40178571428571425
Recall: 0.25
F1 Score: 0.2833594976452119

start RF 10 PGD
Accuracy: 0.23657838224767358

macro
Precision: 0.2363895960214602
Recall: 0.2367948797284276
F1 Score: 0.23642187997287836

weighted
Precision: 0.236332139343917
Recall: 0.23657838224767358
F1 Score: 0.2362849404824325

start RF 10 DF
Accuracy: 0.43222143364088006

macro
Precision: 0.5168942135

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:

# torch.save(model.state_dict(), "./dnn_pytorch_FeatureSqueezing.pt")

In [16]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")